In [ ]:
# notebooks/04_evidently_ai_drift_report.py
# Equivalent of 04_evidently_ai_drift_report.ipynb
import pandas as pd
from evidently import Report, DataDriftPreset

# Create reference (old) and current (new) data
ref_data = {'feature1': range(20), 'feature2': [i * 2 for i in range(20)]}
reference_df = pd.DataFrame(ref_data)

curr_data = {'feature1': range(10, 30), 'feature2': [i * 2.5 for i in range(10, 30)]}
current_df = pd.DataFrame(curr_data)

data_drift_report = Report(metrics=[DataDriftPreset()])
data_drift_report.run(reference_data=reference_df, current_data=current_df)

# Save the report to an HTML file
data_drift_report.save_html("evidently_drift_report.html")
print("Evidently AI drift report saved to evidently_drift_report.html")

